# Testing Notebook

This notebook is for development purposes and includes examples for interacting with Trino through pyhive.

In [73]:
from pyhive import trino
from typing import List
from pydantic import create_model

In [15]:
def get_conn():
    return trino.connect('localhost')
def get_cursor():
    return get_conn().cursor()

In [47]:
cursor = get_cursor()

## Database Introspection

In [48]:
# Get Schemas
def get_schemas(exclude: List[str] = None) -> List[str]:
    
    query_string = "SHOW SCHEMAS"
    cursor.execute(query_string) 
    schemas = [row[0] for row in cursor.fetchall()]
    if exclude is not None:
        schemas = [s for s in schemas if s not in exclude]
    return schemas

In [59]:
def get_tables_in_schema(schema: str, exclude: List[str] = None) -> List[str]:
    
    query_string = f"SHOW TABLES IN {schema}"
    cursor.execute(query_string) 
    tables = [row[0] for row in cursor.fetchall()]
    if exclude is not None:
        tables = [t for t in tables if t not in exclude]
    return tables

In [74]:
def get_cols_in_table(schema:str, table: str) -> dict:
    
    type_mapper = {
        'bigint':int,
        'timestamp(3)': str,
        'double': float,
        'varchar': str
    }
    
    query_string = f"DESC {schema}.{table}"
    cursor.execute(query_string)
    res = cursor.fetchall()
    col_names = [row[0] for row in res]
    col_types = [type_mapper[row[1]] for row in res]
    return dict(zip(col_names, col_types))

In [75]:
def get_schema_structure(schema:str) -> dict:
    
    schema_structure = {}
    tables = get_tables_in_schema(schema=schema)
    for table in tables:
        schema_structure[table] = get_cols_in_table(schema=schema, table=table)
    return schema_structure

In [76]:
def get_db_structure(exclude_schemas: List[str] = None) -> dict:
    
    db_structure = {}
    schemas = get_schemas(exclude = exclude_schemas)
    for schema in schemas:
        db_structure[schema] = get_schema_structure(schema=schema)
    return db_structure

In [77]:
db_structure = get_db_structure(exclude_schemas=["information_schema"])

In [78]:
db_structure

{'default': {},
 'taxi': {'trips': {'vendorid': int,
   'tpep_pickup_datetime': str,
   'tpep_dropoff_datetime': str,
   'passenger_count': float,
   'trip_distance': float,
   'pulocationid': int,
   'dolocationid': int,
   'ratecodeid': float,
   'store_and_fwd_flag': str,
   'dropoff_longitude': float,
   'dropoff_latitude': float,
   'payment_type': int,
   'fare_amount': float,
   'improvement_surcharge': float,
   'congestion_surcharge': float,
   'mta_tax': float,
   'tip_amount': float,
   'tolls_amount': float,
   'extra': float,
   'airport_fee': float,
   'total_amount': float}}}

In [81]:
def generate_api_routes_and_models(db_structure:Dict[str, dict]) ->:
    routes_and_models = {}
    for schema, tables in db_structure.items():
        if tables:
            for table, cols in tables.items():
                route = f"/{schema}/{table}"
                model_name = schema[0].upper()+schema[1:].lower()+table[0].upper()+table[1:].lower()
                model = create_model(model_name, **cols)
                routes_and_models[route] = model
    return routes_and_models

In [87]:
typ = routes_and_models["/taxi/trips"]

In [96]:
typ

pydantic.main.TaxiTrips

In [68]:
# Get Data
query_string = """
SELECT * FROM taxi.trips LIMIT 10
"""

In [97]:
cursor.execute(query_string)
columns = cursor.description
data = cursor.fetchall()
print("Columns:", "\n", columns)
print("Data:" , "\n", data)

Columns: 
 [('vendorid', 'bigint', None, None, None, None, True), ('tpep_pickup_datetime', 'timestamp', None, None, None, None, True), ('tpep_dropoff_datetime', 'timestamp', None, None, None, None, True), ('passenger_count', 'double', None, None, None, None, True), ('trip_distance', 'double', None, None, None, None, True), ('pulocationid', 'bigint', None, None, None, None, True), ('dolocationid', 'bigint', None, None, None, None, True), ('ratecodeid', 'double', None, None, None, None, True), ('store_and_fwd_flag', 'varchar', None, None, None, None, True), ('dropoff_longitude', 'double', None, None, None, None, True), ('dropoff_latitude', 'double', None, None, None, None, True), ('payment_type', 'bigint', None, None, None, None, True), ('fare_amount', 'double', None, None, None, None, True), ('improvement_surcharge', 'double', None, None, None, None, True), ('congestion_surcharge', 'double', None, None, None, None, True), ('mta_tax', 'double', None, None, None, None, True), ('tip_amount

In [100]:
samples = []
for row in data:
    obj = {}
    for i in range(len(columns)):
        obj[columns[i][0]] = row[i]
    samples.append(obj)


In [102]:
import json
json.dumps(samples)

'[{"vendorid": 1, "tpep_pickup_datetime": "2022-01-01 00:35:40.000", "tpep_dropoff_datetime": "2022-01-01 00:53:29.000", "passenger_count": 2.0, "trip_distance": 3.8, "pulocationid": 142, "dolocationid": 236, "ratecodeid": 1.0, "store_and_fwd_flag": "N", "dropoff_longitude": null, "dropoff_latitude": null, "payment_type": 1, "fare_amount": 14.5, "improvement_surcharge": 0.3, "congestion_surcharge": 2.5, "mta_tax": 0.5, "tip_amount": 3.65, "tolls_amount": 0.0, "extra": 3.0, "airport_fee": 0.0, "total_amount": 21.95}, {"vendorid": 1, "tpep_pickup_datetime": "2022-01-01 00:33:43.000", "tpep_dropoff_datetime": "2022-01-01 00:42:07.000", "passenger_count": 1.0, "trip_distance": 2.1, "pulocationid": 236, "dolocationid": 42, "ratecodeid": 1.0, "store_and_fwd_flag": "N", "dropoff_longitude": null, "dropoff_latitude": null, "payment_type": 1, "fare_amount": 8.0, "improvement_surcharge": 0.3, "congestion_surcharge": 0.0, "mta_tax": 0.5, "tip_amount": 4.0, "tolls_amount": 0.0, "extra": 0.5, "airp

In [105]:
inst = typ()

TaxiTrips()